In [19]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd

In [20]:
BASE_URL = "https://paper-api.alpaca.markets"
KEY_ID = "PKV5IDOVGQUJGDEPNJRK"
SECRET_KEY = "7FI0KztqobeHfYK0GKDMAkSnCxMO1M70ug0XNUBc"

In [33]:
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Bitcoin
bars = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df
print(bars)

                          exchange      open      high       low     close  \
timestamp                                                                    
2022-05-17 05:00:00+00:00     CBSE  30336.24  30358.11  30323.81  30323.81   
2022-05-17 05:00:00+00:00     FTXU  30346.00  30346.00  30346.00  30346.00   
2022-05-17 05:01:00+00:00     CBSE  30322.61  30355.23  30322.60  30349.35   
2022-05-17 05:02:00+00:00     CBSE  30347.15  30374.20  30347.15  30368.99   
2022-05-17 05:02:00+00:00     FTXU  30373.00  30375.00  30373.00  30375.00   
...                            ...       ...       ...       ...       ...   
2022-05-17 15:33:00+00:00     CBSE  30021.59  30064.93  30021.59  30064.48   
2022-05-17 15:33:00+00:00     ERSX  30048.80  30048.80  30044.80  30044.80   
2022-05-17 15:33:00+00:00     FTXU  30021.00  30065.00  30021.00  30064.00   
2022-05-17 15:34:00+00:00     CBSE  30064.48  30142.23  30064.47  30137.62   
2022-05-17 15:34:00+00:00     FTXU  30068.00  30136.00  30068.00

In [34]:
# Filter data by exchange
bars = bars[bars.exchange == 'CBSE']
print(bars)

                          exchange      open      high       low     close  \
timestamp                                                                    
2022-05-17 05:00:00+00:00     CBSE  30336.24  30358.11  30323.81  30323.81   
2022-05-17 05:01:00+00:00     CBSE  30322.61  30355.23  30322.60  30349.35   
2022-05-17 05:02:00+00:00     CBSE  30347.15  30374.20  30347.15  30368.99   
2022-05-17 05:03:00+00:00     CBSE  30368.76  30378.63  30365.26  30365.27   
2022-05-17 05:04:00+00:00     CBSE  30365.27  30365.27  30323.61  30328.87   
...                            ...       ...       ...       ...       ...   
2022-05-17 15:30:00+00:00     CBSE  29960.14  30006.66  29952.66  30001.56   
2022-05-17 15:31:00+00:00     CBSE  29998.49  30030.13  29998.49  30000.00   
2022-05-17 15:32:00+00:00     CBSE  30000.01  30021.59  29991.22  30021.59   
2022-05-17 15:33:00+00:00     CBSE  30021.59  30064.93  30021.59  30064.48   
2022-05-17 15:34:00+00:00     CBSE  30064.48  30142.23  30064.47

In [35]:
# Create a market order to buy 1 Bitcoin
order_buy = api.submit_order('BTCUSD', qty=1, side='buy')

In [ ]:
# Create a market order to sell 1 Bitcoin
#order_sell = api.submit_order('BTCUSD', qty=1, side='sell')
order_sell = api.submit_order('DOGEUSD', qty=1, side='sell')

In [25]:
# Get current position on Bitcoin. Yields error if we don't hold any
api.get_position('BTCUSD')

# Get all positions and find out if we hold any Bitcoin
positions = api.list_positions()
position_qty = 0
for p in positions:
    if p.symbol == 'BTCUSD':
        position_qty = float(p.qty)
print(position_qty)

2.0


In [ ]:
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

In [ ]:
import random
import time

SYMBOL = 'BTCUSD'
while True:
    # GET OUR CURRENT POSITION
    position = get_position(symbol=SYMBOL)
    
    # SCIENTIFICALLY CHECK IF WE SHOULD BUY OR SELL
    gods_say_buy = random.choice([True, False])
    print(f"Holding: {position} / Gods: {gods_say_buy}")

    #CHECK IF WE SHOULD BUY
    if position == 0 and gods_say_buy == True:
        # WE BUY ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='buy')
    #HECK IF WE SHOULD SELL
    elif position > 0 and gods_say_buy == False:
        # WE SELL ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='sell')
    print('Lets wait for the gods to manifest again...')
    print("*"*20)
    time.sleep(10)

Holding: 1.0 / Gods: True
Lets wait for the gods to manifest again...
********************


In [32]:
from datetime import datetime, timedelta
import math
import time

SYMBOL = 'BTCUSD'
SMA_FAST = 12
SMA_SLOW = 24
QTY_PER_TRADE = 1


# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars = bars[bars.exchange == 'CBSE']
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    print(f"Position: {position} / Should Buy: {should_buy}")
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')

    time.sleep(get_pause())
    print("*"*20)

Fast 30006.91083333334  /  Slow: 30029.818333333333
Position: 0 / Should Buy: False
Sleep for 18
********************
Fast 30004.974166666667  /  Slow: 30026.74208333333
Position: 0 / Should Buy: False
Sleep for 58
********************
Fast 29999.579166666663  /  Slow: 30023.572083333333
Position: 0 / Should Buy: False
Sleep for 59
********************
Fast 29997.115  /  Slow: 30021.437083333334
Position: 0 / Should Buy: False
Sleep for 58


KeyboardInterrupt: 